In [4]:
import pandas as pd 
import numpy as np 
from sklearn.preprocessing import StandardScaler 
from sklearn import preprocessing 
import itertools 
import signal
from random import seed
from random import randrange
from sklearn.preprocessing import LabelEncoder
import scipy.stats
from math import sqrt
import os

from Errors import Error_Metrics

import matplotlib as mpl
import pylab
import matplotlib.pyplot as plt
mpl.rcParams['lines.linewidth'] = 6
mpl.rcParams['lines.color'] = 'r'
mpl.rcParams['font.weight'] = 200
plt.style.use('seaborn-whitegrid')
plt.rc('figure',figsize=(11,6))
pylab.rcParams['ytick.major.pad']='12'
pylab.rcParams['xtick.major.pad']='12'
mpl.axes.Axes.annotate

mpl.rcParams['axes.labelsize'] = 22
mpl.rcParams['axes.linewidth'] = 4
mpl.rcParams['xtick.labelsize'] = 28
mpl.rcParams['ytick.labelsize'] = 28
mpl.rcParams['axes.edgecolor'] = 'black'
mpl.rcParams['axes.titlesize'] = 25
mpl.rcParams['legend.fontsize'] = 25

# Reading Data 

In [ ]:
training_person = ['Player']
training_sensor = ['Right_Leg','Left_Leg']  

In [ ]:
training_data,training_label = extract_data(training_person,training_sensor)

# Activity Label

In [ ]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(training_label)
training_y = label_encoder.transform(training_label)

In [ ]:
individual_label.shape

In [ ]:
training_y.shape

In [ ]:
training_data = pd.DataFrame(StandardScaler().fit_transform(training_data.reshape(training_data.shape[0],-1)))

In [ ]:
testing_data  = pd.DataFrame(StandardScaler().fit_transform(testing_data.reshape(testing_data.shape[0],-1)))

In [ ]:
training_data.shape

In [ ]:
training_data['label']=pd.DataFrame(training_y)
testing_data['label']=pd.DataFrame(testing_y)

In [5]:
if not os.path.exists('Error Learning Results'):
    os.makedirs('Error Learning Results')

# Distance-based Error Learning Framework

In [ ]:
batch_size = 30
flag = 0
K = 25
for label in np.unique(training_y):
    temp10 = testing_data[testing_data['label']==label].values
    temp12 = training_data[training_data['label']==label].values
    temp10 = temp10[:,0:-1]
    temp12 = temp12[:,0:-1]        
    temp13 = np.tile(temp12,(batch_size,1,1))
    print(label)
    for i in range(0, temp10.shape[0], batch_size):
        temp11 = temp10[i:i+batch_size,:]
        if temp11.shape[0]<batch_size:
            #temp13 = np.tile(temp12,(temp11.shape[0],1,1))
            temp13 = temp13[0:temp11.shape[0],:,:]
        distance = (np.square(temp11[:,np.newaxis,:])+np.square(temp13)- 2*np.multiply(temp11[:,np.newaxis,:],temp13))
        distance = np.sum(distance,axis=2)
        idx = np.argpartition(distance,-K,axis=1)[:,-K:]
        averaged_chunk = np.average(temp12[idx,:],axis=1)
        if flag==0:
            averaged_data=averaged_chunk
            preserved_label = np.full((averaged_chunk.shape[0],1),label)
            flag+=1
        else: 
            averaged_data=np.append(averaged_data,averaged_chunk,axis=0)
            preserved_label = np.append(preserved_label,np.full((averaged_chunk.shape[0],1),label))
    preserved_label = np.reshape(preserved_label,(len(preserved_label),1))    

In [ ]:
temp_averaged_data = averaged_data
averaged_data = pd.DataFrame(averaged_data)
averaged_data['label'] = pd.DataFrame(preserved_label)

In [ ]:
averaged_data.shape

In [ ]:
testing_data.shape

# Saving as .csv file

In [ ]:
training_person= ['Player']
testing_person= ['Player']

In [ ]:
averaged_data.to_csv("Error Learning Results/averaged_data_k_25_"+training_person+"_"+testing_person".csv",index=False,header=None)

In [ ]:
testing_data.to_csv("Error Learning Results/testing_data_"+training_person+"_"+testing_person".csv",index=False,header=None)

# Handcrafted Errors

In [ ]:
K=25

In [ ]:
avg_data = pd.read_csv("Error Learning Results/averaged_data_k_25_"+training_person+"_"+testing_person".csv",header=None)
avg_data=np.array(avg_data)
testing_data = pd.read_csv("Error Learning Results/testing_data_"+training_person+"_"+testing_person".csv",header=None)
testing_data=np.array(testing_data)

In [ ]:
testing_data.shape

In [ ]:
avg_data.shape

In [ ]:
error_metrics = Error_Metrics(avg_data,testing_data)

In [ ]:
error_metrics = pd.DataFrame(error_metrics)
eror_metrics.to_csv("Error Learning Results/error_metrics_k_"+str(K)+"_"+training_person+"_"+testing_person".csv",index=False,header=None)

# Completed the handcrafted Errors Computing

# Plotting the error as Probability Density Function

In [ ]:
list(label_encoder.inverse_transform([0]))

In [ ]:
xlabels = ['RMSE','MSE','NMSE','MAE','MdAE','RMDSPE','NRMSE_Sd','SSE','ED_L2_Norm','NRMSE_m','MPE','MNB','MdSPE']

# Reading the error csv files and plotting them

In [6]:
if not os.path.exists('Error Learning Results/Error Figures'):
    os.makedirs('Error Learning Results/Error Figures')

In [ ]:
training_person= ['Player']
testing_person= ['Player']

In [ ]:
error_player = pd.read_csv("Error Learning Results/error_metrics_k_25__"+training_person+"_"+testing_person".csv",header=None)
error_player= np.array(error_player)
error_player1 = pd.read_csv("Error Learning Results/error_metrics_k_25__"+training_person+"_"+testing_person".csv",header=None)
error_player1 = np.array(error_player1)
error_player2 = pd.read_csv("Error Learning Results/error_metrics_k_25__"+training_person+"_"+testing_person".csv",header=None)
error_player2 = np.array(error_player2)


for i in np.arange(0,13):
    for j in np.arange(0,12):
        error_player_label= error_player[error_player[:,13]==j,i]
        hist_val_error_player=np.histogram(error_player_label,100)
        plt.plot(hist_val_error_player[1][:-1],hist_val_error_player[0]/sum(hist_val_error_player[0]),'r')
        
        error_player1_label= error_player1[error_player1[:,13]==j,i]
        hist_val_error_player1=np.histogram(error_player1_label,100)
        plt.plot(hist_val_error_player1[1][:-1],hist_val_error_player1[0]/sum(hist_val_error_player1[0]),'g')
        
        error_player2_label= error_player2[error_player2[:,13]==j,i]
        hist_val_error_player2=np.histogram(error_player2_label,100)
        plt.plot(hist_val_error_player2[1][:-1],hist_val_error_player2[0]/sum(hist_val_error_player2[0]),'k')
        
        
        #plt.title(list(label_encoder.inverse_transform([j]))[0],fontweight='semibold', fontsize = '25')
        plt.xlabel(xlabels[i],fontweight='semibold', fontsize = '30')
        plt.ylabel('Normalized Frequency', fontweight='semibold', fontsize = '30')
        plt.legend(['Professional Player 1', 'Novice Player', 'Intermediate Player'],prop=dict(weight='semibold'))
        plt.tight_layout()
        
        
        plt.savefig("Error Learning Results//Error Figures/K_"+str(K)+"/"+list(label_encoder.inverse_transform([j]))[0]+"_"+xlabels[i]+".eps", format='eps')
        plt.show()  
    